# Scraping the best routes for each destination from Wikiloc.

Now that we have all mountain passes (both its name and coordinates) we will begin by scraping all road bike routes on **Wikiloc** that match the name of a mountain pass. Our tool of choice will be the web scraping engine **Selenium**.

In [1]:
#Importing libraries.

import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [2]:
#Initializing our webdriver (Chrome).

driver = webdriver.Chrome()

In [3]:
#Logging into Wikiloc.

def login():
    time.sleep(1)
    driver.get('https://es.wikiloc.com/wikiloc/start.do')
    time.sleep(1)
    login_button = driver.find_element_by_class_name('form-control')
    login_button.click()
    login_button.send_keys('username')
    password_button = driver.find_element_by_xpath('//*[@id="password"]')
    password_button.click()
    password_button.send_keys('password')

In [5]:
#At this point we MUST solve the captcha manually. Once solved we can proceed.

In [4]:
#Navigating to the search section with our "road bike" filter activated.

def filtro():
    search_bar = driver.find_element_by_xpath('/html/body/header/nav/div/div[1]/div/div')
    search_bar.click()
    time.sleep(1)
    search_bar_open = driver.find_element_by_xpath('/html/body/header/nav/div/div[1]/div/div[1]/input')
    search_bar_open.send_keys('bracons')
    time.sleep(1)
    bracons = driver.find_element_by_xpath('/html/body/header/nav/div/div[1]/div/div[2]/div/ul/div')
    bracons.click()
    time.sleep(1)
    actividad = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[1]/div/div[1]/button')
    actividad.click()
    time.sleep(1)
    selector_actividad = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[1]/div/div[1]/div/div/div/div[1]/div[1]/div/div[2]/div/div[2]/div')
    selector_actividad.click()
    time.sleep(1)
    aplicar_filtro = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[1]/div/div[1]/div/div/div/div[2]/button')
    aplicar_filtro.click() #Selecting the "road bike" filter. Now we're all set to scrape any destination.

In [5]:
#This function scrapes all relevant data from the found routes and returns it as a list of dictionaries.

def scraper_ruta(puerto):
    listado_dict = []
    element = driver.find_elements_by_class_name('trail-list__wrapper')
    for i in range(len(element)):
        try:
            element[i].click()
            time.sleep(0.3)
            nombre_ruta = driver.find_elements_by_xpath('//*[@id="front"]/div/div/h1')[0].text
            url = driver.current_url
            trailrank = driver.find_elements_by_xpath('//*[@id="front"]/div/div/a')[0].text
            distancia = driver.find_elements_by_xpath('//*[@id="trail-data"]/div[1]/div[1]/a/span[2]')[0].text
            desnivel = driver.find_elements_by_xpath('//*[@id="trail-data"]/div[1]/div[3]/a/span[2]')[0].text
            dificultad = driver.find_elements_by_xpath('//*[@id="trail-data"]/div[1]/div[7]/a/span[2]')[0].text
            fotos = driver.find_elements_by_class_name('trail-photo')
            photo1 = fotos[0].get_attribute('href')
            photo2 = fotos[1].get_attribute('href')
            photo3 = fotos[2].get_attribute('href')
            driver.back()
            time.sleep(0.3)
            element = driver.find_elements_by_class_name('trail-list__wrapper')
            dict_ruta = {'ubicacion': puerto, 'nombre': nombre_ruta, 'trailrank': trailrank,'distancia': distancia, 'desnivel': desnivel,
                         'dificultad': dificultad, 'url': url, 'photo1': photo1, 'photo2': photo2, 'photo3': photo3}
            listado_dict.append(dict_ruta)
        except:
            pass
            driver.back()
            time.sleep(0.5)
            element = driver.find_elements_by_class_name('trail-list__wrapper')
    return listado_dict

In [16]:
#Defining a function that will search routes for the requested destination.

def buscar_ubicacion(ubicacion):
    time.sleep(0.3)
    driver.find_elements_by_xpath('/html/body/header/nav/div/div[1]/div/div/input')[0].click()
    time.sleep(0.3)
    driver.find_elements_by_xpath('/html/body/header/nav/div/div[1]/div/div[1]/input')[0].send_keys(ubicacion)
    time.sleep(0.5)
    driver.find_element_by_xpath('/html/body/header/nav/div/div[1]/div/div[2]/div/ul/div').click()
    time.sleep(0.5)

In [7]:
df_puertos = pd.read_csv('df_puertos_final.csv')

In [8]:
df_puertos.head()

,Unnamed: 0,puerto,provincia,pueblo,altitud,desnivel,distancia,pendiente,coeficiente,url
0,0,Pico Veleta,Granada,El Purche-Sabinas-Pradollano,3384,2662,46.0,5.7,588,https://www.altimetrias.net/aspbk/verPerfilusu...
1,1,Iram,Granada,Haza Llana,2845,2080,35.0,5.8,546,https://www.altimetrias.net/aspbk/verPerfilusu...
2,2,Angliru,Asturias,Santa Eulalia,1570,1423,18.0,7.0,528,https://www.altimetrias.net/aspbk/verPuerto.as...
3,3,Ancares Desde Navia De Suarna,Lugo,Pan do Zarco,1670,1875,35.0,3.8,512,https://www.altimetrias.net/aspbk/verPerfilusu...
4,4,Gamoniteiro,Asturias,Pola-Cobertoria,1772,1465,15.0,9.0,492,https://www.altimetrias.net/aspbk/verPuerto.as...


In [9]:
#Creating a list containing all mountain pass names.

lista_ubicaciones = df_puertos['puerto']
lista_ubicaciones = lista_ubicaciones.to_list()

In [11]:
len(lista_ubicaciones)

1723

## Final function

In [12]:
#This function incorporates all previous functions, iterating through each destination and returning a dataframe of routes.

def scraper_puertos(puertoncios):
    start = time.time()
    lista = []
    for i in puertoncios:
        try:
            buscar_ubicacion(i)
            lista.append(scraper_ruta(i))
        except:
            pass
    df = pd.DataFrame(lista[0])
    for i in lista[1:]:
        df = df.append(i, ignore_index=True)
    stop = time.time() 
    duration = (stop - start) / 60
    print('Minutos:', duration)
    return df

In [18]:
#Benchmarking our function.

df_test = scraper_puertos(lista_ubicaciones[])

Minutos: 6.923399611314138


In [ ]:
#Creating a dataframe of routes using the function.

df_test.to_csv('df_test.csv', index=False)

In [19]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ubicacion   86 non-null     object
 1   nombre      86 non-null     object
 2   trailrank   86 non-null     object
 3   distancia   86 non-null     object
 4   desnivel    86 non-null     object
 5   dificultad  86 non-null     object
 6   url         86 non-null     object
 7   photo1      86 non-null     object
 8   photo2      86 non-null     object
 9   photo3      86 non-null     object
dtypes: object(10)
memory usage: 6.8+ KB


## Cleaning our dataframe

Now that we've obtained a dataframe of our routes, some data cleaning is needed.

In [6]:
df1 = pd.read_csv('df_0_50.csv') #Loading all obtained csv files as not to re-run the code.
df2 = pd.read_csv('df_50_100.csv')
df3 = pd.read_csv('df_100_150.csv')
df4 = pd.read_csv('df_150_end.csv')

In [11]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True) #Uniting them in a single DF.

In [14]:
df.drop('Unnamed: 0', inplace=True, axis=1) #Dropping a trash column.

In [19]:
df.head()

,ubicacion,nombre,trailrank,distancia,desnivel,dificultad,url,photo1,photo2,photo3
0,SIERRA NEVADA-PICO VELETA,Día 1/2 - Sierra Nevada - Granada - Pico Veleta,64,"108,78 km",2.914 m,2,https://es.wikiloc.com/rutas-ciclismo/dia-1-2-...,https://es.wikiloc.com/rutas-ciclismo/dia-1-2-...,https://es.wikiloc.com/rutas-ciclismo/dia-1-2-...,https://es.wikiloc.com/rutas-ciclismo/dia-1-2-...
1,SIERRA NEVADA-PICO VELETA,Cenes de la vega-Sierra nevada-pico veleta-cen...,43,"82,83 km",2.779 m,2,https://es.wikiloc.com/rutas-ciclismo/cenes-de...,https://es.wikiloc.com/rutas-ciclismo/cenes-de...,https://es.wikiloc.com/rutas-ciclismo/cenes-de...,https://es.wikiloc.com/rutas-ciclismo/cenes-de...
2,SIERRA NEVADA-PICO VELETA,Granada- El Purche- Güejar Sierra- Haza Llanas...,35,"107,41 km",3.341 m,4,https://es.wikiloc.com/rutas-ciclismo/granada-...,https://es.wikiloc.com/rutas-ciclismo/granada-...,https://es.wikiloc.com/rutas-ciclismo/granada-...,https://es.wikiloc.com/rutas-ciclismo/granada-...
3,SIERRA NEVADA-PICO VELETA,Pinos genil - station Sierra nevada - pico de ...,32,"125,46 km",2.130 m,4,https://es.wikiloc.com/rutas-ciclismo/pinos-ge...,https://es.wikiloc.com/rutas-ciclismo/pinos-ge...,https://es.wikiloc.com/rutas-ciclismo/pinos-ge...,https://es.wikiloc.com/rutas-ciclismo/pinos-ge...
4,SIERRA NEVADA-PICO VELETA,Sierra Nevada hoya de la mora/ Radio telescopi...,20,"32,53 km",1.201 m,4,https://es.wikiloc.com/rutas-ciclismo/sierra-n...,https://es.wikiloc.com/rutas-ciclismo/sierra-n...,https://es.wikiloc.com/rutas-ciclismo/sierra-n...,https://es.wikiloc.com/rutas-ciclismo/sierra-n...


In [20]:
df.to_csv('routes.csv')

In [18]:
#Assigning a numerical score to the route difficulty.

df.loc[df['dificultad'] == 'Fácil', 'dificultad'] = 1
df.loc[df['dificultad'] == 'Moderado', 'dificultad'] = 2
df.loc[df['dificultad'] == 'Difícil', 'dificultad'] = 3
df.loc[df['dificultad'] == 'Muy difícil', 'dificultad'] = 4
df.loc[df['dificultad'] == 'Sólo expertos', 'dificultad'] = 5

In [23]:
#We see that we've got 526 locations, 49 less than in our 'altimetrias' dataset. This means that 49 locations didn't have
#routes matching our quality criteria.

df['ubicacion'].value_counts() 

EL MEDIANO                   24
SAN JUAN DE ARTZUAGA         24
MAS DE LA COSTA              24
PAJARES                      24
LA PUEBLA                    24
                             ..
BERMIEGO                      1
LEABURU                       1
URRUZTIMENDI                  1
VALLENÓN-CAMPO LAS DANZAS     1
EMBALSE DE ARTIBA             1
Name: ubicacion, Length: 526, dtype: int64

In [31]:
df.sort_values(by=['trailrank'], ascending=False).head(50)

,ubicacion,nombre,trailrank,distancia,desnivel,dificultad,url,photo1,photo2,photo3
1583,LA MINA,San francisco circular en bici,99,"42,05 km",528 m,2,https://es.wikiloc.com/rutas-ciclismo/san-fran...,https://es.wikiloc.com/rutas-ciclismo/san-fran...,https://es.wikiloc.com/rutas-ciclismo/san-fran...,https://es.wikiloc.com/rutas-ciclismo/san-fran...
5168,BECI,San francisco circular en bici,99,"42,05 km",528 m,2,https://es.wikiloc.com/rutas-ciclismo/san-fran...,https://es.wikiloc.com/rutas-ciclismo/san-fran...,https://es.wikiloc.com/rutas-ciclismo/san-fran...,https://es.wikiloc.com/rutas-ciclismo/san-fran...
1584,LA MINA,Guarne - Llanogrande - La Ceja - Rionegro - Gu...,97,"69,53 km",738 m,2,https://es.wikiloc.com/rutas-ciclismo/guarne-l...,https://es.wikiloc.com/rutas-ciclismo/guarne-l...,https://es.wikiloc.com/rutas-ciclismo/guarne-l...,https://es.wikiloc.com/rutas-ciclismo/guarne-l...
1585,LA MINA,CORONANDO el GARBI por la FRONTERA desde Valen...,96,69 km,787 m,2,https://es.wikiloc.com/rutas-ciclismo/coronand...,https://es.wikiloc.com/rutas-ciclismo/coronand...,https://es.wikiloc.com/rutas-ciclismo/coronand...,https://es.wikiloc.com/rutas-ciclismo/coronand...
1587,LA MINA,La Cabra por Almuñecar - 8-11-2012 - Granada,95,"159,35 km",2.296 m,2,https://es.wikiloc.com/rutas-ciclismo/la-cabra...,https://es.wikiloc.com/rutas-ciclismo/la-cabra...,https://es.wikiloc.com/rutas-ciclismo/la-cabra...,https://es.wikiloc.com/rutas-ciclismo/la-cabra...
1586,LA MINA,"Iznalloz, Pto. Zegri, Colomera - 18-09-2014",95,"97,4 km",942 m,1,https://es.wikiloc.com/rutas-ciclismo/iznalloz...,https://es.wikiloc.com/rutas-ciclismo/iznalloz...,https://es.wikiloc.com/rutas-ciclismo/iznalloz...,https://es.wikiloc.com/rutas-ciclismo/iznalloz...
1588,LA MINA,Villanueva Mesia - 17-02-2015,94,"101,46 km",609 m,2,https://es.wikiloc.com/rutas-ciclismo/villanue...,https://es.wikiloc.com/rutas-ciclismo/villanue...,https://es.wikiloc.com/rutas-ciclismo/villanue...,https://es.wikiloc.com/rutas-ciclismo/villanue...
948,EL ESCUDO,Benalua de las Villas - 4-09-2014,93,"76,12 km",814 m,1,https://es.wikiloc.com/rutas-ciclismo/benalua-...,https://es.wikiloc.com/rutas-ciclismo/benalua-...,https://es.wikiloc.com/rutas-ciclismo/benalua-...,https://es.wikiloc.com/rutas-ciclismo/benalua-...
1591,LA MINA,"Granada, Guadix y vuelta por Lugros - 5-06-2014",93,"140,26 km",1.832 m,2,https://es.wikiloc.com/rutas-ciclismo/granada-...,https://es.wikiloc.com/rutas-ciclismo/granada-...,https://es.wikiloc.com/rutas-ciclismo/granada-...,https://es.wikiloc.com/rutas-ciclismo/granada-...
1590,LA MINA,Benalua de las Villas - 4-09-2014,93,"76,12 km",814 m,1,https://es.wikiloc.com/rutas-ciclismo/benalua-...,https://es.wikiloc.com/rutas-ciclismo/benalua-...,https://es.wikiloc.com/rutas-ciclismo/benalua-...,https://es.wikiloc.com/rutas-ciclismo/benalua-...


**<div align="right">Ironhack DA PT 2021</div>**
    
**<div align="right">Xavier Esteban</div>**